# Power and Connected UEs over Time

This notebook loads:
- Power per PCI over time from `cell_power_changes.csv` and computes a Total Power across PCIs.
- Connected UEs over time from `input.csv`.

It then aligns both series on a common time axis and plots them with dual y-axes.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Seaborn style and fonts
sns.set_theme(
    style="whitegrid",
    font_scale=1.5
)

# File paths
base = Path('/home/vmadmin/Energy-Saver-Tests/results/energy-over-time')
file_power = base / 'cell_power_changes_20250816_144851_final.csv'
file_ues = base / 'input.csv'

file_power, file_ues

In [ ]:
# Load data
power_df = pd.read_csv(file_power)
ues_df = pd.read_csv(file_ues)

# Inspect and normalize column names
power_df.columns = [c.strip() for c in power_df.columns]
ues_df.columns = [c.strip() for c in ues_df.columns]

# Rename time columns for consistency
if 'Time(h)' in power_df.columns:
    power_df = power_df.rename(columns={'Time(h)': 'time_h'})
if 'Time(h)' in ues_df.columns:
    ues_df = ues_df.rename(columns={'Time(h)': 'time_h'})

# Compute total power across all PCI columns
pci_cols = [c for c in power_df.columns if c.startswith('pci_')]
power_df['total_power'] = power_df[pci_cols].sum(axis=1, skipna=True)

# Compute number of active PCIs per timestamp (non-zero entries across pci_* columns)
if pci_cols:
    power_df['active_pci'] = (power_df[pci_cols].abs() > 1e-9).sum(axis=1)
else:
    power_df['active_pci'] = 0

# Keep only time, total power, and active PCI count
power_df = power_df[['time_h', 'total_power', 'active_pci']].sort_values('time_h').reset_index(drop=True)

# Keep only time and connected UEs
ue_col_candidates = [c for c in ues_df.columns if 'UE' in c or 'Connected' in c]
ue_col = ue_col_candidates[0] if ue_col_candidates else ues_df.columns[1]
ues_df = ues_df[['time_h', ue_col]].rename(columns={ue_col: 'connected_ues'})
ues_df = ues_df.sort_values('time_h').reset_index(drop=True)

power_df.head(), ues_df.head(), ue_col

In [ ]:
# Align on time
# We'll do an asof merge to interpolate UEs at power timestamps (or vice versa)
# Create uniformly sampled time axis covering both ranges
min_t = min(power_df['time_h'].min(), ues_df['time_h'].min())
max_t = max(power_df['time_h'].max(), ues_df['time_h'].max())

# Resolution: choose the finer of the two mean spacings
power_dt = np.diff(power_df['time_h']).mean() if len(power_df) > 1 else 0.01
ues_dt = np.diff(ues_df['time_h']).mean() if len(ues_df) > 1 else 0.01
step = max(min(power_dt, ues_dt), 0.001)

grid = pd.DataFrame({'time_h': np.arange(min_t, max_t + step/2, step)})

# Interpolate each series onto the grid
power_grid = pd.merge_asof(grid, power_df.sort_values('time_h'), on='time_h')
ues_grid = pd.merge_asof(grid, ues_df.sort_values('time_h'), on='time_h')

merged = grid.join(power_grid[['total_power','active_pci']], how='left')
merged = merged.join(ues_grid[['connected_ues']], how='left')

# Optional: linear interpolation for smoother plots
merged['total_power'] = merged['total_power'].interpolate()
merged['connected_ues'] = merged['connected_ues'].interpolate()

# For active_pci, we typically want step-like values; forward fill then backfill to fill gaps
merged['active_pci'] = merged['active_pci'].ffill().bfill()

# Plot with seaborn/matplotlib (style like Plot3)
plt.figure(figsize=(14, 6))
ax1 = plt.gca()
ax2 = ax1.twinx()
# Remove gridlines for the secondary axis
ax2.grid(False)

# Primary lines
sns.lineplot(data=merged, x='time_h', y='total_power', ax=ax1, color='tab:blue', label='Total Power')
sns.lineplot(data=merged, x='time_h', y='connected_ues', ax=ax2, color='tab:red', label='Connected UEs')

# Add hourly bars for active PCI count on a third axis sharing x
ax3 = ax1.twinx()
# Offset ax3 spine to the right (moved a little farther)
ax3.spines.right.set_position(("axes", 1.1))
ax3.set_frame_on(True)
ax3.patch.set_visible(False)
# Choose a neutral color for bars
hour_start = np.floor(merged['time_h'].min())
hour_end = np.ceil(merged['time_h'].max())
hours = np.arange(hour_start, hour_end, 1.0)
bar_centers = hours + 0.5
bar_width = 0.8
# Compute mean active_pci per hour bin
hour_bins = pd.cut(merged['time_h'], bins=np.append(hours, hour_end), right=False, include_lowest=True)
active_per_hour = merged.groupby(hour_bins)['active_pci'].mean().reindex(pd.IntervalIndex.from_breaks(np.append(hours, hour_end), closed='left'))
bar_vals = active_per_hour.values if active_per_hour is not None else np.array([])
# Align centers for bins
centers = hours + 0.5

bars = ax3.bar(centers, bar_vals, width=bar_width, color='gray', alpha=0.25, label='Active PCI (avg/hr)')
ax3.set_ylim(0, max(5, np.nanmax(bar_vals) * 1.3))
ax3.set_ylabel('Active PCI (avg/hr)', color='gray')
# Hide grid for ax3
ax3.grid(False)

# Annotate bars with integer rounded counts centered within the bar
for x, v in zip(centers, bar_vals):
    if np.isfinite(v) and v > 0:
        ax3.text(x, v/3.0, f"{int(round(v))}", ha='center', va='center', fontsize=10, color='black')

ax1.set_xlabel('Time(h)')
# Integer ticks like Plot3
try:
    xmin, xmax = float(merged['time_h'].min()), float(merged['time_h'].max())
    ax1.set_xticks(np.arange(int(np.floor(xmin)), int(np.ceil(xmax)) + 1, 1))
except Exception:
    pass

ax1.set_ylabel('Total Power (dbm)', color='tab:blue')
ax2.set_ylabel('Connected UEs', color='tab:red')
#plt.title('Total Power and Connected UEs over Time')

# Combined legend (single box)
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles3, labels3 = ax3.get_legend_handles_labels()
# Remove any per-axis legends
for ax in (ax1, ax2, ax3):
    leg = ax.get_legend()
    if leg is not None:
        leg.remove()
# Build combined lists and drop no-legend entries
handles = []
labels = []
for h, l in list(zip(handles1, labels1)) + list(zip(handles2, labels2)) + list(zip(handles3, labels3)):
    if not l or l == '_nolegend_' or l in labels:
        continue
    handles.append(h)
    labels.append(l)
ax1.legend(handles, labels, loc='upper left', frameon=True)



plt.tight_layout()
# Save like Plot3
out_dir = base / 'out'
out_dir.mkdir(parents=True, exist_ok=True)

plt.savefig(out_dir / 'power_vs_ues.png', bbox_inches='tight')
plt.savefig(out_dir / 'power_vs_ues.pdf', dpi=300, bbox_inches='tight')
plt.show()

## Notes
- X-axis is hours from the CSVs.
- Total Power is the sum across all `pci_*` columns.
- Connected UEs is taken from the `Connected UEs` column of `input.csv`.
- The series are interpolated onto a common grid for a smooth comparison.

## Zoomed view

In [ ]:
# Filter interval and plot
start_h, end_h = 18 + 55/60, 20 + 5/60  # 18:55 to 20:05
context_pad = 0.0  # no extra context
view_start, view_end = start_h, end_h

# Data slices
focus = merged[(merged['time_h'] >= start_h) & (merged['time_h'] <= end_h)].copy()

plt.figure(figsize=(14, 6))
ax1 = plt.gca()
ax2 = ax1.twinx()
# Ensure no gridlines on ax2
ax2.grid(False)

# Emphasized focus lines (18:55–20:05)
sns.lineplot(data=focus, x='time_h', y='total_power', ax=ax1, color='tab:blue', linewidth=2, label='Total Power')
sns.lineplot(data=focus, x='time_h', y='connected_ues', ax=ax2, color='tab:red', linewidth=2, label='Connected UEs')

ax1.set_xlabel('Time(h)')
ax1.set_xlim(view_start, view_end)
# Remove vertical gridlines on primary axis only
ax1.grid(False, axis='x')

# Optimization bars only in 19:00–20:00
opt_window_start, opt_window_end = 19.0, 20.0

# Add optimization intervals (orange) and following handover intervals (green) using provided durations
handover_label_positions = []  # (center_time, duration_seconds)
optimization_label_positions = []  # (center_time, duration_seconds)
try:
    # Editable lists (seconds)
    opt_durations_sec = [3.7, 4.2, 4.4, 4.0]
    handover_durations_sec = [475, 550, 575, 525]
    # Minimum visual width for orange intervals (in hours). 10 seconds by default.
    min_opt_width_hours = 10.0 / 3600.0
    anchors = np.linspace(opt_window_start, opt_window_end, 4, endpoint=False)
    for i, t_anchor in enumerate(anchors):
        if opt_durations_sec and i < 4:  # safeguard
            opt_dur_sec = float(opt_durations_sec[i % len(opt_durations_sec)])
            desired_width_hours = max(opt_dur_sec / 3600.0, min_opt_width_hours)
            half = desired_width_hours / 2.0
            t0 = t_anchor - half
            t1 = t_anchor + half
            # Constrain to window
            if t0 < opt_window_start:
                shift = opt_window_start - t0
                t0 += shift
                t1 += shift
            if t1 > opt_window_end:
                shift = t1 - opt_window_end
                t0 -= shift
                t1 -= shift
            # Final clamp
            t0 = max(t0, opt_window_start)
            t1 = min(t1, opt_window_end)
            # Ensure still positive width
            if t1 > t0:
                ax1.axvspan(t0, t1, facecolor='tab:orange', alpha=0.6, zorder=0)
                optimization_label_positions.append(((t0 + t1) / 2.0, opt_dur_sec))
                # Add matching handover interval immediately after optimization
                if handover_durations_sec:
                    ho_dur_sec = float(handover_durations_sec[i % len(handover_durations_sec)])
                    ho_dur_hours = ho_dur_sec / 3600.0
                    ho_start = t1
                    ho_end = min(ho_start + ho_dur_hours, opt_window_end)
                    if ho_end > ho_start:
                        ax1.axvspan(ho_start, ho_end, facecolor='tab:green', alpha=0.3, zorder=0)
                        center_time = (ho_start + ho_end) / 2.0
                        handover_label_positions.append((center_time, ho_dur_sec))
except Exception:
    pass

# Cap to 4 optimization labels in case of any future changes
optimization_label_positions = optimization_label_positions[:4]

# X ticks as HH:MM at 15-minute intervals starting at 19:00
tick_start = max(19.0, view_start)
ticks = np.arange(tick_start, view_end + 1e-9, 0.25)
ax1.set_xticks(ticks)
labels = []
for t in ticks:
    h = int(np.floor(t))
    m = int(round((t - h) * 60))
    if m == 60:
        h += 1
        m = 0
    labels.append(f"{h:02d}:{m:02d}")
ax1.set_xticklabels(labels, rotation=45, ha='right')

# Title helper (commented out)
def _hhmm(x):
    h = int(np.floor(x))
    m = int(round((x - h) * 60))
    if m == 60:
        h += 1
        m = 0
    return f"{h:02d}:{m:02d}"
# plt.title(f"Total Power and Connected UEs over Time ({_hhmm(start_h)}–{_hhmm(end_h)})")

ax1.set_ylabel('Total Power (dbm)', color='tab:blue')
ax2.set_ylabel('Connected UEs', color='tab:red')

# Helper to format durations with higher precision
def format_duration(d):
    if d < 60:
        return f"{d:.1f}s"  # seconds with one decimal
    m = int(d // 60)
    s = d - 60 * m
    return f"{m}m{s}s"  # minutes plus seconds (raw seconds remainder)

# After setting lines, get y-limits
ymin, ymax = ax1.get_ylim()
y_range = ymax - ymin

# Annotate optimization (orange) intervals with arrows
opt_target_y = ymin + 0.85 * y_range
opt_text_y = ymin + 0.93 * y_range
for i, (center_time, dur_s) in enumerate(optimization_label_positions):
    opt_label = format_duration(dur_s)
    dx = 0.015 * (1 if i % 2 == 0 else -1)  # alternate left/right text anchor
    text_x = center_time + dx
    text_x = max(view_start + 0.001, min(view_end - 0.001, text_x))
    ha = 'left' if dx > 0 else 'right'
    ax1.annotate(opt_label,
                 xy=(center_time, opt_target_y), xycoords='data',
                 xytext=(text_x, opt_text_y), textcoords='data',
                 arrowprops=dict(arrowstyle='->', color='black', lw=1),
                 ha=ha, va='bottom', fontsize=12, fontweight='bold', color='black',
                 bbox=dict(boxstyle='round,pad=0.2', facecolor='white', edgecolor='none', alpha=0.75))

# Annotate handover (green) intervals with inline labels near bottom
y_text = ymin + 0.15 * y_range  # 15% from bottom
for center_time, dur_s in handover_label_positions:
    label = format_duration(dur_s)
    ax1.text(center_time, y_text, label, ha='center', va='center', fontsize=12,
             color='black', fontweight='bold', alpha=0.9,
             bbox=dict(boxstyle='round,pad=0.2', facecolor='white', edgecolor='none', alpha=0.7))

# Combined legend (single box) with patches for intervals
from matplotlib.patches import Patch
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
for ax in (ax1, ax2):
    leg = ax.get_legend()
    if leg is not None:
        leg.remove()
handles, labels = [], []
for h, l in list(zip(h1, l1)) + list(zip(h2, l2)):
    if not l or l == '_nolegend_' or l in labels:
        continue
    handles.append(h)
    labels.append(l)
opt_patch = Patch(facecolor='tab:orange', label='Optimization Interval')
ho_patch = Patch(facecolor='tab:green', alpha=0.3, label='Handover Interval')
if 'Optimization Interval' not in labels:
    handles.append(opt_patch)
    labels.append('Optimization Interval')
if 'Handover Interval' not in labels:
    handles.append(ho_patch)
    labels.append('Handover Interval')
ax1.legend(handles, labels, loc='upper right', frameon=True)

plt.tight_layout()
# Ensure output directory exists and save zoomed plot
out_dir = base / 'out'
out_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(out_dir / f'power_vs_ues_{start_h}_{end_h}.png', bbox_inches='tight')
plt.savefig(out_dir / f'power_vs_ues_{start_h}_{end_h}.pdf', dpi=300, bbox_inches='tight')
plt.show()